# Agents and their tools

In [10]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]

chat_openai = ChatOpenAI(openai_api_key=openai_api_key, model = "gpt-4o-mini", temperature=0)

# Function Calling

### Easy tools

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    print("Multiplying two numbers")
    return a * b

llm_with_tools = chat_openai.bind_tools([multiply])

### Important: Tools-Aware Agent returns only call to the function, nothing more

1. Function which may help solving the problem
2. Parses parameters for such function

Only then we can run such call

In [3]:
from langchain_core.messages import HumanMessage

tool_call = llm_with_tools.invoke([HumanMessage(content="What is two multiplied by three?", name="Mr")])
tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SZlcHHomNwmaffyrkbJlkyts', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 61, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_86d0290411', 'id': 'chatcmpl-BJ7ACVRUn1DMdXm87euBxAXuwrkTP', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9acede0a-7e1c-4469-bfaa-480d7dd47b89-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_SZlcHHomNwmaffyrkbJlkyts', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61, 'output_tokens': 18, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

In [ ]:
# To jeszcze nie jest wywołanie tej funkcji!
tool_call.additional_kwargs['tool_calls']

[{'id': 'call_SZlcHHomNwmaffyrkbJlkyts',
  'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'},
  'type': 'function'}]

In [ ]:
# Without using an additional model
import json

tool_call_data = tool_call.additional_kwargs['tool_calls'][0]
print(f"tool_call_data value: {tool_call_data}")

function_name = tool_call_data['function']['name']
print(f"function_name value: {function_name}")

function_args = json.loads(tool_call_data['function']['arguments'])
print(f"function_args value: {function_args}")

# Ręczne wrzucenie argumentów do funkcji
if function_name == "multiply":
    result = multiply(**function_args)
    print(f"Multiplication result: {result}")

tool_call_data value: {'id': 'call_SZlcHHomNwmaffyrkbJlkyts', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}
function_name value: multiply
function_args value: {'a': 2, 'b': 3}
Multiplying two numbers
Multiplication result: 6


In [6]:
import json

tool_call = llm_with_tools.invoke([HumanMessage(content="What is two hundred twenty two times seventy seven?", name="Mr")])

tool_call_data = tool_call.additional_kwargs['tool_calls'][0]

function_name = tool_call_data['function']['name']

function_args = json.loads(tool_call_data['function']['arguments'])

if function_name == "multiply":
    result = multiply(**function_args)
    print(f"Multiplication result: {result}")

Multiplying two numbers
Multiplication result: 17094


# More advanced approach - integrate with external tools

### Get weather information from Open Weather Map

https://openweathermap.org

In [12]:
# Simple service call
from langchain_community.utilities import OpenWeatherMapAPIWrapper
weather_wrapper = OpenWeatherMapAPIWrapper()
weather_wrapper.run("Warsaw")

'In Warsaw, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 8.75 m/s, direction: 340°\nHumidity: 61%\nTemperature: \n  - Current: 0.9°C\n  - High: 1.51°C\n  - Low: 0.36°C\n  - Feels like: -5.43°C\nRain: {}\nHeat index: None\nCloud cover: 75%'

### Use whole chain - this will be easier with langgraph

In [13]:
# Create Tool-Aware Agent
from langchain import hub
from langchain.agents import load_tools, AgentExecutor, create_tool_calling_agent

weather_tools = load_tools(["openweathermap-api"])

# Previous approach
llm_with_weather = chat_openai.bind_tools(weather_tools)
llm_with_weather.invoke([HumanMessage(content="What is the weather in Warsaw?", name="Mr")])

# Use dedicated prompt from public library
prompt = hub.pull("hwchase17/openai-tools-agent")
print(f"prompt value: {prompt}")

# Creating our own Agent Executor to make the action
agent = create_tool_calling_agent(chat_openai, weather_tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=weather_tools, verbose=True)

# Wreszcie możemy zadać pytanie agentowi
agent_executor.invoke(
    {
        "input": "What is the temperature of the Los Angeles?",
    }
)

/Users/tomasz/Developer/TEG---Technologie-Generatywne/.venv/lib/python3.11/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


prompt value: input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemM

{'input': 'What is the temperature of the Los Angeles?',
 'output': 'The current temperature in Los Angeles is 22.6°C. It feels like 21.7°C, with a high of 25.73°C and a low of 19.66°C.'}

### Add internet browser: Tavily

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults(max_results=5)
tools = [tavily, weather_tools[0]]
agent = create_tool_calling_agent(chat_openai, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke(
    {
        "input": "What is the square root of current temperature in Warsaw multiplied by root of -1?",
    }
)



> Entering new AgentExecutor chain...

Invoking: `open_weather_map` with `{'location': 'Warsaw'}`


In Warsaw, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 8.75 m/s, direction: 340°
Humidity: 61%
Temperature: 
  - Current: 0.9°C
  - High: 1.51°C
  - Low: 0.36°C
  - Feels like: -5.43°C
Rain: {}
Heat index: None
Cloud cover: 75%The current temperature in Warsaw is 0.9°C. 

To find the square root of the current temperature multiplied by the square root of -1, we can express this mathematically:

1. The square root of 0.9 is approximately \( \sqrt{0.9} \).
2. The square root of -1 is represented as \( i \) in complex numbers.

Thus, the expression becomes:

\[
\sqrt{0.9} \times i
\]

Calculating \( \sqrt{0.9} \):

\[
\sqrt{0.9} \approx 0.9487
\]

Therefore, the result is approximately:

\[
0.9487i
\]

So, the square root of the current temperature in Warsaw multiplied by the square root of -1 is approximately \( 0.9487i \).

> Finished chain.


### Add Wikipedia search

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [18]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
data = wikipedia.run("Wrocław")
print(data)

Page: Wrocław
Summary: Wrocław is a city in southwestern Poland and the largest city and historical capital of the region of Silesia. It lies on the banks of the Oder River in the Silesian Lowlands of Central Europe, roughly 40 kilometres (25 mi) from the Sudeten Mountains to the south. The city consists of 48 districts. As of 2023, the official population of Wrocław is 674,132 making it the third largest city in Poland. The population of the Wrocław metropolitan area is around 1.25 million.
Wrocław is the historical capital of Silesia and Lower Silesia. Today, it is the capital of the Lower Silesian Voivodeship. The history of the city dates back over 1,000 years; at various times, it has been part of the Duchy of Poland, Kingdom of Poland, the Kingdom of Bohemia, the Kingdom of Hungary, the Habsburg monarchy of Austria, the Kingdom of Prussia and Germany, until it became again part of Poland in 1945 as the result of territorial changes of Poland immediately after World War II.
Wrocła

In [21]:
tools = [tavily, weather_tools[0], wikipedia]
agent = create_tool_calling_agent(chat_openai, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke(
    {
        "input": "Tell me something about mbank",
    }
)

print(result)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'mbank'}`


Page: MBank
Summary: mBank SA (formerly BRE Bank), set up in 1986, and originally BRE – Bank Rozwoju Eksportu (Export Development Bank), is Poland's fourth largest universal banking group in terms of total assets and loans, and fifth by deposits at the end of September 2016. It offers retail, corporate and investment banking as well as other financial services such as leasing, factoring, insurance, financing of commercial real property, brokerage operations, wealth management, corporate finance and advisory in the scope of capital markets.
Since 1992, it has been listed on the Warsaw Stock Exchange. In 2007 it started its retail operations in the Czech Republic and Slovakia.
The current mBank CEO is Cezary Stypułkowski.

Page: Bank One Corporation
Summary: Bank One Corporation was an American bank founded in 1968 and at its peak the sixth-largest bank in the United States. It traded on the New Yo

In [22]:
tools = [tavily, weather_tools[0], wikipedia]
agent = create_tool_calling_agent(chat_openai, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke(
    {
        "input": "What is the square root of the temperature of the city where Techland was created?",
    }
)

print(result)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Techland'}`


Page: Techland
Summary: Techland S.A. is a Polish video game developer and publisher founded in 1991 by Paweł Marchewka. It developed Call of Juarez (2006) and its prequel Call of Juarez: Bound in Blood (2009), as well as Dead Island (2011) and the Dying Light series. The company is headquartered in Wrocław in western Poland and has offices in Wrocław and Warsaw.

Page: List of Techland video games
Summary: This is a list of Techland video games.

Page: Dying Light
Summary: Dying Light is a 2015 survival horror video game developed by Techland and published by Warner Bros. Interactive Entertainment. The game's story follows undercover agent Kyle Crane who is sent to infiltrate a quarantine zone in a fictional Middle Eastern city called Harran. It features an enemy-infested, open world city with a dynamic day–night cycle, in which zombies are slow and clumsy during daytime and extremely aggress

# Finally - research papers search

Let's firstly try without arxiv tool

In [23]:
result = agent_executor.invoke(
    {
        "input": "Give me links to 5 newest publications about reinforcement machine learning in computer games industry",
    }
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'reinforcement learning in computer games industry'}`


[{'title': '[PDF] The Application of Reinforcement Learning in Video Games', 'url': 'https://www.atlantis-press.com/article/125994505.pdf', 'content': 'rein-forcement learning, this study highlights the potential benefits it brings to the video game industry. Furthermore, real-world cases are examined to showcase successful applications of reinforcement learning in video games. It concludes that the future and advancement of reinforcement learning in video games hold great promise. Keywords: Reinforcement Learning, Video Games Playing, Video Game De-velopment 1 Introduction The reinforcement learning (RL) has emerged as an important method of [...] 3.2 The RL Based Artificial Intelligence in Games RL, a type of machine learning that has rapidly gained popularity in the game industry, unlike traditional machine AI implementations, RL all

Now we can add him arxiv search possibility

In [24]:
arxiv = load_tools(["arxiv"])[0]
tools = [tavily, wikipedia, arxiv]

In [26]:
agent = create_tool_calling_agent(chat_openai, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke(
    {
        "input": "What is the newest research articles from arxiv within data science in telco industry?",
    }
)



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'data science telco industry'}`


Published: 2021-08-24
Title: Context-aware Telco Outdoor Localization
Authors: Yige Zhang, Weixiong Rao, Mingxuan Yuan, Jia Zeng, Pan Hui
Summary: Recent years have witnessed the fast growth in telecommunication (Telco)
techniques from 2G to upcoming 5G. Precise outdoor localization is important
for Telco operators to manage, operate and optimize Telco networks. Differing
from GPS, Telco localization is a technique employed by Telco operators to
localize outdoor mobile devices by using measurement report (MR) data. When
given MR samples containing noisy signals (e.g., caused by Telco signal
interference and attenuation), Telco localization often suffers from high
errors. To this end, the main focus of this paper is how to improve Telco
localization accuracy via the algorithms to detect and repair outlier positions
with high errors. Specifically, we propose a context-aware Telco 